#  Linear advection equation

This file simulates the problem shown in Figure 6.3.

In [ ]:
%config InlineBackend.figure_format = 'svg'
from clawpack import pyclaw
from clawpack import riemann
import numpy as np
import matplotlib.pyplot as plt

This function calls clawpack to solve the problem.

In [ ]:
def solve(ncell=200, order=1, limiter=0):
    claw = pyclaw.Controller()
    claw.tfinal = 10.0
    claw.keep_copy = True       # Keep solution data in memory for plotting
    claw.output_format = None   # Don't write solution data to file
    claw.num_output_times = 50  # Write 50 output frames
    claw.verbosity = 0

    riemann_solver = riemann.advection_1D
    claw.solver = pyclaw.ClawSolver1D(riemann_solver)
    claw.solver.all_bcs = pyclaw.BC.periodic
    claw.solver.dt_initial = 1.e99
    claw.solver.order = order
    claw.solver.limiters = limiter

    xmin, xmax = 0.0, 1.0
    domain = pyclaw.Domain( (xmin,), (xmax,), (ncell,))

    claw.solution = pyclaw.Solution(claw.solver.num_eqn, domain)

    x = domain.grid.x.centers
    beta, x0, freq = 200.0, 0.5, 80.0
    claw.solution.q[0, :] = np.exp(-beta * (x-x0)**2) * np.sin(freq * x)

    claw.solution.state.problem_data['u'] = 1.0

    status = claw.run()

    return claw


Plot initial solution and at a later time.

In [ ]:
def make_plots(claw):
    x = claw.grid.x.centers
    q0 = claw.frames[0].q[0, :]

    plt.figure(figsize=(10, 5))

    plt.subplot(121)
    f = 5
    q = claw.frames[f].q[0, :]
    plt.plot(x, q0, '-', label='Exact')
    plt.plot(x, q, 'o', fillstyle='none', label='Numerical')
    plt.axis([0, 1, -1.0, 1.0])
    plt.title('Time = ' + str(claw.frames[f].t))
    plt.legend()

    plt.subplot(122)
    f = 25
    q = claw.frames[f].q[0, :]
    plt.plot(x, q0, '-', label='Exact')
    plt.plot(x, q, 'o', fillstyle='none', label='Numerical')
    plt.axis([0, 1, -1.0, 1.0])
    plt.title('Time = ' + str(claw.frames[f].t))
    plt.legend()

## First order (upwind) scheme

In [ ]:
claw = solve(order=1, limiter=0)
make_plots(claw)

## Lax-Wendroff without limiter

In [ ]:
claw = solve(order=2, limiter=0)
make_plots(claw)

## Lax-Wendroff with minmod limiter

In [ ]:
claw = solve(order=2, limiter=pyclaw.limiters.tvd.minmod)
make_plots(claw)

## Lax-Wendroff with superbee limiter

In [ ]:
claw = solve(order=2, limiter=pyclaw.limiters.tvd.superbee)
make_plots(claw)

## Lax-Wendroff with MC limiter

In [ ]:
claw = solve(order=2, limiter=pyclaw.limiters.tvd.MC)
make_plots(claw)

To examine the evolution more thoroughly, it's nice to see all the frames in sequence.  We can do this as follows.

In [ ]:
from clawpack.visclaw import ianimate
ianimate.ianimate(claw)